# 2A. Prediksi Position


In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import precision_score, \
    recall_score, classification_report, \
    accuracy_score, f1_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [2]:
players_df = pd.read_csv("datasets/players.csv",";")
data_df = pd.read_csv("datasets/all_seasons.csv")

C:\Users\frigas\AppData\Local\Temp\ipykernel_4224\1908320892.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  players_df = pd.read_csv("datasets/players.csv",";")


## Preprocessing data dan menggabungkan sebuah dataset lain

In [3]:
display(players_df)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Alaa Abdelnaby,PF,22,POR,43,0,6.7,1.3,2.7,...,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1991
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19,22.5,6.2,15.1,...,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1991
2,3,Mark Acres,C,28,ORL,68,0,19.3,1.6,3.1,...,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1991
3,4,Michael Adams,PG,28,DEN,66,66,35.5,8.5,21.5,...,0.9,3,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1991
4,5,Mark Aguirre,SF,31,DET,78,13,25.7,5.4,11.7,...,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18039,536,Delon Wright,PG,28,SAC,27,8,25.8,3.9,8.3,...,1,2.9,3.9,3.6,1.6,0.4,1.3,1.1,10,2021
18040,537,Thaddeus Young,PF,32,CHI,68,23,24.3,5.4,9.7,...,2.5,3.8,6.2,4.3,1.1,0.6,2,2.2,12.1,2021
18041,538,Trae Young,PG,22,ATL,63,63,33.7,7.7,17.7,...,0.6,3.3,3.9,9.4,0.8,0.2,4.1,1.8,25.3,2021
18042,539,Cody Zeller,C,28,CHO,48,21,20.9,3.8,6.8,...,2.5,4.4,6.8,1.8,0.6,0.4,1.1,2.5,9.4,2021


In [4]:
data_df.rename(columns = {'player_name':'Player', 'season':'Year'}, inplace = True)
data_df["Year"] = data_df["Year"].str[:4]



data_df = data_df.loc[:,["Player", "Year", "player_height", "player_weight"]]

display(data_df)

,Player,Year,player_height,player_weight
0,Dennis Rodman,1996,198.12,99.790240
1,Dwayne Schintzius,1996,215.90,117.933920
2,Earl Cureton,1996,205.74,95.254320
3,Ed O'Bannon,1996,203.20,100.697424
4,Ed Pinckney,1996,205.74,108.862080
...,...,...,...,...
11140,Maxi Kleber,2019,208.28,108.862080
11141,Melvin Frazier Jr.,2019,195.58,97.522280
11142,Meyers Leonard,2019,213.36,117.933920
11143,Norvel Pelle,2019,208.28,104.779752


In [5]:
players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18044 entries, 0 to 18043
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rk      18044 non-null  object
 1   Player  18044 non-null  object
 2   Pos     18044 non-null  object
 3   Age     18044 non-null  object
 4   Tm      18044 non-null  object
 5   G       18044 non-null  object
 6   GS      18044 non-null  object
 7   MP      18044 non-null  object
 8   FG      18044 non-null  object
 9   FGA     18044 non-null  object
 10  FG%     18044 non-null  object
 11  3P      18044 non-null  object
 12  3PA     18044 non-null  object
 13  3P%     18044 non-null  object
 14  2P      18044 non-null  object
 15  2PA     18044 non-null  object
 16  2P%     18044 non-null  object
 17  eFG%    18044 non-null  object
 18  FT      18044 non-null  object
 19  FTA     18044 non-null  object
 20  FT%     18044 non-null  object
 21  ORB     18044 non-null  object
 22  DRB     18044 non-null

In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11145 entries, 0 to 11144
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Player         11145 non-null  object 
 1   Year           11145 non-null  object 
 2   player_height  11145 non-null  float64
 3   player_weight  11145 non-null  float64
dtypes: float64(2), object(2)
memory usage: 348.4+ KB


In [7]:
print("Jumlah duplikasi data pada dataset mvps:", players_df.duplicated().sum(), "maka lakukan drop data terlebih dahulu")

players_df = players_df.drop_duplicates(inplace = False)
print("Jumlah duplikasi data setelah drop duplikasinya:", players_df.duplicated().sum())

Jumlah duplikasi data pada dataset mvps: 627 maka lakukan drop data terlebih dahulu
Jumlah duplikasi data setelah drop duplikasinya: 0


In [8]:
players_df.drop(players_df.index[players_df['Rk'] == 'Rk'], inplace = True)

In [9]:
data_df["Year"] = pd.to_numeric(data_df["Year"])

In [10]:
players_df = pd.merge(data_df, players_df, on=["Player", "Year"])

display(players_df)

,Player,Year,player_height,player_weight,Rk,Pos,Age,Tm,G,GS,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Dwayne Schintzius,1996,215.90,117.933920,342,C,27,IND,33,8,...,619,0.7,1.7,2.4,0.4,0.3,0.4,0.6,1.6,3.4
1,Ed O'Bannon,1996,203.20,100.697424,272,SG,23,NJN,64,29,...,713,1,1.6,2.6,1,0.7,0.2,1,1.5,6.2
2,Ed Pinckney,1996,205.74,108.862080,296,PF,32,TOT,74,47,...,0.76,2.6,3.6,6.2,1,0.9,0.4,1,2.1,6.5
3,Ed Pinckney,1996,205.74,108.862080,296,PF,32,TOR,47,24,...,758,2.4,3.6,6,1.1,0.7,0.4,1.1,2.1,7
4,Ed Pinckney,1996,205.74,108.862080,296,PF,32,PHI,27,23,...,764,2.7,3.8,6.5,0.8,1.2,0.4,0.9,2,5.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9782,Matthew Dellavedova,2019,190.50,90.718400,133,PG,28,MIL,12,0,...,1,0,0.8,0.8,2.4,0.2,0,0.9,0.8,1.7
9783,Matthew Dellavedova,2019,190.50,90.718400,133,PG,28,CLE,36,0,...,792,0.2,1.7,1.9,4.2,0.3,0.1,1.6,1.8,7.3
9784,Maurice Harkless,2019,200.66,99.790240,208,SF,25,POR,60,53,...,671,1.3,3.2,4.5,1.2,1.1,0.9,0.8,2.7,7.7
9785,Maxi Kleber,2019,208.28,108.862080,283,PF,27,DAL,71,18,...,784,1.3,3.4,4.6,1,0.5,1.1,0.8,2,6.8


## Pemisahan kolom-kolom menjadi feature (X) dan target (y) dan drop fitur-fitur yang tidak diperlukan

In [11]:
players_df = players_df.drop(["Rk", "Player", "Tm", "Year"], axis=1)

display(players_df.columns)

lst_column = players_df.columns

for i in lst_column : 
    if i != "Pos":
        players_df[i] = pd.to_numeric(players_df[i])

display(players_df.info())

lst_pos = players_df["Pos"].unique()

Index(['player_height', 'player_weight', 'Pos', 'Age', 'G', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9787 entries, 0 to 9786
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_height  9787 non-null   float64
 1   player_weight  9787 non-null   float64
 2   Pos            9787 non-null   object 
 3   Age            9787 non-null   int64  
 4   G              9787 non-null   int64  
 5   GS             9787 non-null   int64  
 6   MP             9787 non-null   float64
 7   FG             9787 non-null   float64
 8   FGA            9787 non-null   float64
 9   FG%            9787 non-null   float64
 10  3P             9787 non-null   float64
 11  3PA            9787 non-null   float64
 12  3P%            9787 non-null   float64
 13  2P             9787 non-null   float64
 14  2PA            9787 non-null   float64
 15  2P%            9787 non-null   float64
 16  eFG%           9787 non-null   float64
 17  FT             9787 non-null   float64
 18  FTA     

None

In [12]:
players_df = players_df.loc[(players_df["Pos"] == "SG") | (players_df["Pos"] == "PG") | (players_df["Pos"] == "SF") | (players_df["Pos"] == "PF")| (players_df["Pos"] == "C")]

display(players_df["Pos"].unique())



array(['C', 'SG', 'PF', 'SF', 'PG'], dtype=object)

In [13]:
players_df["Pos"] = players_df["Pos"].replace(['PF', 'PG', 'C', 'SF', 'SG'], [0,1,2,3,4])

display(players_df)


players_df["Pos"] = pd.to_numeric(players_df["Pos"])

,player_height,player_weight,Pos,Age,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,215.90,117.933920,2,27,33,8,9.0,1.5,3.3,445.00,...,619.00,0.7,1.7,2.4,0.4,0.3,0.4,0.6,1.6,3.4
1,203.20,100.697424,4,23,64,29,19.6,2.4,6.3,0.39,...,713.00,1.0,1.6,2.6,1.0,0.7,0.2,1.0,1.5,6.2
2,205.74,108.862080,0,32,74,47,23.1,2.3,4.5,0.51,...,0.76,2.6,3.6,6.2,1.0,0.9,0.4,1.0,2.1,6.5
3,205.74,108.862080,0,32,47,24,21.9,2.5,5.0,502.00,...,758.00,2.4,3.6,6.0,1.1,0.7,0.4,1.1,2.1,7.0
4,205.74,108.862080,0,32,27,23,25.1,2.0,3.8,529.00,...,764.00,2.7,3.8,6.5,0.8,1.2,0.4,0.9,2.0,5.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9782,190.50,90.718400,1,28,12,0,8.1,0.5,1.6,316.00,...,1.00,0.0,0.8,0.8,2.4,0.2,0.0,0.9,0.8,1.7
9783,190.50,90.718400,1,28,36,0,19.9,2.6,6.2,413.00,...,792.00,0.2,1.7,1.9,4.2,0.3,0.1,1.6,1.8,7.3
9784,200.66,99.790240,3,25,60,53,23.6,3.2,6.5,487.00,...,671.00,1.3,3.2,4.5,1.2,1.1,0.9,0.8,2.7,7.7
9785,208.28,108.862080,0,27,71,18,21.2,2.5,5.4,453.00,...,784.00,1.3,3.4,4.6,1.0,0.5,1.1,0.8,2.0,6.8


In [14]:
X = players_df.drop('Pos', axis=1)
y = players_df['Pos'].to_frame()



In [15]:
from sklearn.model_selection import train_test_split

# Membagi dataset menjadi training set dan test set
# dengan jumlah test set adalah sebanyak 20% dari data keseluruhan 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Membagi dataset train menjadi training set dan test set
# dengan jumlah test set adalah sebanyak 20% dari data train

X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

# define the models and the parameter grids
models = [RandomForestClassifier(), LogisticRegression(),SVC(), MLPClassifier(), KNeighborsClassifier(), GaussianNB() ]
param_grids = [{
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
},
               {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']},
               {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
               {'hidden_layer_sizes': [(10,), (50,), (100,)], 'learning_rate_init': [0.1, 0.01, 0.001], 'activation' : ['relu', 'logistic']},
               {'n_neighbors': [1,2,3,4,5, 6, 7, 8, 9], 'metric': ['euclidean', 'manhattan']},
               {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4]}]

# create the grid search objects
grid_searches = []
for model, param_grid in zip(models, param_grids):
    grid_search = GridSearchCV(model, param_grid, cv=5)
    grid_searches.append(grid_search)

# fit the grid search objects to the training data
for grid_search in grid_searches:
    grid_search.fit(X_train_temp, y_train_temp)

# print the best parameters and the best scores for each model
for i, grid_search in enumerate(grid_searches):
    print(f"Model {i}:")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best score: {grid_search.best_score_}")

# retrain the models with the best parameters on the entire training set
best_models = []
for grid_search in grid_searches:
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_temp, y_train_temp)
    best_models.append(best_model)

# evaluate the models on the test set
for i, model in enumerate(best_models):
    test_score = model.score(X_test_temp, y_test_temp)
    print(f"Model {i}: Test score = {test_score}")


c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

Model 0:
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best score: 0.7941794664510914
Model 1:
Best parameters: {'C': 10, 'penalty': 'l2'}
Best score: 0.7689571544058206
Model 2:
Best parameters: {'C': 10, 'kernel': 'linear'}
Best score: 0.7710590137429264
Model 3:
Best parameters: {'activation': 'logistic', 'hidden_layer_sizes': (10,), 'learning_rate_init': 0.01}
Best score: 0.770250606305578
Model 4:
Best parameters: {'metric': 'manhattan', 'n_neighbors': 9}
Best score: 0.6596604688763137
Model 5:
Best parameters: {'var_smoothing': 0.0001}
Best score: 0.6674211802748585


c:\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, w

Model 0: Test score = 0.7789269553975436
Model 1: Test score = 0.7575953458306399
Model 2: Test score = 0.768584356819651
Model 3: Test score = 0.7705235940530059
Model 4: Test score = 0.649644473173885
Model 5: Test score = 0.6302521008403361


Dari GridCV diatas diketahui model terbaik adalah dengan menggunakan random forest dengan parameter {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}. Lalu kita uji dengan data test asli yang kita siapkan

In [18]:
# evaluate the models on the test set
for i, model in enumerate(best_models):
    if i == 0 :
        predicted = model.predict(X_test)

        # Menampilkan metrics
        print("Nilai rata-rata akurasi:", model.score(X_test, y_test))
        print('F1 Macro Average:', f1_score(y_test, predicted, average='macro'))
        print('F1 Micro Average:', f1_score(y_test, predicted, average='micro'))
        print('Precision Macro Average:', precision_score(y_test, predicted, average='macro',zero_division=0))
        print('Precision Micro Average:', precision_score(y_test, predicted, average='micro',zero_division=0))
        print('Recall Macro Average:', recall_score(y_test, predicted, average='macro',zero_division=0))
        print('Recall Micro Average:', recall_score(y_test, predicted, average='micro',zero_division=0))
        break



Nilai rata-rata akurasi: 0.8009307135470527
F1 Macro Average: 0.8002120561607138
F1 Micro Average: 0.8009307135470526
Precision Macro Average: 0.8004874644204358
Precision Micro Average: 0.8009307135470527
Recall Macro Average: 0.8004429459228561
Recall Micro Average: 0.8009307135470527


Dengan demikian akurasi yang didapatkan pada final test adalah 80%